In [1]:
import malaya
import json
from tqdm import tqdm

/home/ubuntu/.local/lib/python3.7/site-packages/malaya_boilerplate/frozen_graph.py:25: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  'Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0'


In [2]:
with open('twitter.json') as fopen:
    data = json.load(fopen)
    
len(data)

11

In [3]:
with open('twitter-negeri.json') as fopen:
    data_negeri = json.load(fopen)
    
len(data_negeri)

12

In [4]:
strings = []
for d in data:
    strings.extend(list(d.keys()))
    
for d in data_negeri:
    strings.extend([i['retweet_text_full'] for i in data_negeri[d]])
len(strings)

1864533

In [5]:
normalizer = malaya.normalize.normalizer()

In [6]:
normalizer.normalize(strings[0],
                    normalize_entity = False,
                    normalize_year = False,
                    normalize_telephone = False)

{'normalize': 'Yaysan Islam pon buat kerja balak juga ke ? Memang dah tiada hasil lain dah ke ? ? ? Punggung betui lah',
 'date': {},
 'money': {}}

In [7]:
fast_text = malaya.language_detection.fasttext()

In [8]:
strings[:12]

['Yaysan Islam pon buat kerja balak gak ke? Memang dah takde hasil lain dah ke ??? Pungkoq betui lah',
 'Geleng kepala aku tengok poster matchday Sri Pahang ni. Sangat memalukan untuk pasukan yg main di Liga Super Malaysia dgn bajet puluh juta.\n\nAku sertakan sekali 3 contoh poster matchday dari pasukan amatur yg takde bajet langsung untuk team media, tapi ada jiwa bila buat kerja. https://t.co/zIVL7DZrLT',
 'Aku syak pengadil ni dulu bekas player bola. Dia main striker dulu cita2 nak main Liga Perdana tapi takde rezeki. Tu yang dia geram masuk kotak penalti nak join Kelantan attack cari gol',
 'Kolumpo mana aje lah best dok subang takde apa lak. Aduh',
 'Takde ke pompuan twit "gais ada laki gaduh gais"',
 'Takde aktiviti lain ke siak musim hujan ceni pergi ride kakakaka.',
 'Mungkin semua fikir Covid dah takdee ',
 'Suriahijabs ada banyak lagi jenis inner tudung tau. Get yours here:\n https://t.co/wSXxSXVBrO\n\nTakde Shopee? Whatsapp je\n https://t.co/E4uWDIE2ZD\n(COD Subang Jaya) ht

In [9]:
%%time
fast_text.predict(strings[:1])[0]

CPU times: user 685 µs, sys: 0 ns, total: 685 µs
Wall time: 662 µs


'malay'

In [10]:
def loop(texts):
    texts = texts[0]
    results = []
    for i in tqdm(range(len(texts))):
        if fast_text.predict([texts[i]])[0] not in ['malay', 'rojak']:
            continue
        try:
            r = normalizer.normalize(texts[i],
                    normalize_entity = False,
                    normalize_year = False,
                    normalize_telephone = False)
            results.append((texts[i], r['normalize']))
        except:
            pass
    return results

In [11]:
import mp
processed = mp.multiprocessing(strings, loop)

100%|██████████| 310755/310755 [08:39<00:00, 598.62it/s] 


In [13]:
'\n'.join(processed[0])

'Yaysan Islam pon buat kerja balak gak ke? Memang dah takde hasil lain dah ke ??? Pungkoq betui lah\nYaysan Islam pon buat kerja balak juga ke ? Memang dah tiada hasil lain dah ke ? ? ? Punggung betui lah'

In [14]:
processed[1]

('Geleng kepala aku tengok poster matchday Sri Pahang ni. Sangat memalukan untuk pasukan yg main di Liga Super Malaysia dgn bajet puluh juta.\n\nAku sertakan sekali 3 contoh poster matchday dari pasukan amatur yg takde bajet langsung untuk team media, tapi ada jiwa bila buat kerja. https://t.co/zIVL7DZrLT',
 'Geleng kepala aku tengok poster matchday Sri Pahang ini . Sangat memalukan untuk pasukan yang main di Liga Super Malaysia dengan bajet puluh juta . Aku sertakan sekali tiga contoh poster matchday dari pasukan amatur yang tiada bajet langsung untuk team media , tapi ada jiwa bila buat kerja . https://t.co/zIVL7DZrLT')

In [15]:
len(processed)

689290

In [16]:
with open('twitter-normalized.json', 'w') as fopen:
    json.dump(processed, fopen)

In [11]:
import re

c = lambda x: re.sub(r'[ ]+', ' ', x.replace('\n', ' ')).strip()

In [12]:
!head -n 10 norm-0.txt

Yaysan Islam pon buat kerja balak gak ke? Memang dah takde hasil lain dah ke ??? Pungkoq betui lah
Yaysan Islam pon buat kerja balak juga ke ? Memang dah tiada hasil lain dah ke ? ? ? Punggung betui lah

Geleng kepala aku tengok poster matchday Sri Pahang ni. Sangat memalukan untuk pasukan yg main di Liga Super Malaysia dgn bajet puluh juta. Aku sertakan sekali 3 contoh poster matchday dari pasukan amatur yg takde bajet langsung untuk team media, tapi ada jiwa bila buat kerja. https://t.co/zIVL7DZrLT
Geleng kepala aku tengok poster matchday Sri Pahang ini . Sangat memalukan untuk pasukan yang main di Liga Super Malaysia dengan bajet puluh juta . Aku sertakan sekali tiga contoh poster matchday dari pasukan amatur yang tiada bajet langsung untuk team media , tapi ada jiwa bila buat kerja . https://t.co/zIVL7DZrLT

Aku syak pengadil ni dulu bekas player bola. Dia main striker dulu cita2 nak main Liga Perdana tapi takde rezeki. Tu yang dia geram masuk kotak penalti nak join Kelantan attack

In [17]:
from glob import glob

norms = glob('norm-*.txt')
len(norms)

53

In [32]:
all_data = []

for n in norms:
    with open(n) as fopen:
        data = fopen.read().split('\n')
        data = [data[i: i + 2] for i in range(0, len(data), 3)]
        data = [i for i in data if len(i) > 1]
    all_data.extend(data)
    
len(all_data)

524344

In [33]:
all_data[:100]

[['@izhvns Happy birthday Izhan! Semoga terus berjaya dalam kehidupan hang. btw bila nak masuk minang aku HAHAHAHAHHAHAHAH',
  '@izhvns Gembira birthday Izhan ! Semoga terus berjaya dalam kehidupan hang . btw bila nak masuk minang aku HAHA'],
 ['KSP biadap! #berantasseparatisteroris #papuakujagadankubela #PapuaIndonesia https://t.co/bUFU1SDFEW',
  'KSP biadap ! #berantasseparatisteroris #papuakujagadankubela #PapuaIndonesia https://t.co/bUFU1SDFEW'],
 ['sdgkn boss aku sendiri suruh aku masok siang jdi boleh balik siang lol',
  'sedangkan boss aku sendiri suruh aku masok siang jadi boleh balik siang haha'],
 ['Ketum PB PGRI #berantasseparatisteroris #PapuaIndonesia #papuakujagadankubela https://t.co/V5LFbTZNGf',
  'Ketum PB PGRI #berantasseparatisteroris #PapuaIndonesia #papuakujagadankubela https://t.co/V5LFbTZNGf'],
 ['My teacher once cursed at someone with aku doakan nanti kalau dah beranak, anak tu merangkak masuk balik Now I https://t.co/eGkHRUz9FX',
  'My teacher once cursed at so